In [ ]:
####################################
#ENVIRONMENT SETUP

In [ ]:
#Importing Libraries
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib.ticker as ticker
import matplotlib.cm as cm
from matplotlib.colors import Normalize
from matplotlib.ticker import MaxNLocator
from matplotlib.ticker import ScalarFormatter
import matplotlib.gridspec as gridspec
import xarray as xr
import os; import time
import pickle
os.environ["HDF5_USE_FILE_LOCKING"] = "FALSE"
import h5py

import sys
from tqdm import tqdm

In [ ]:
#MAIN DIRECTORIES
def GetDirectories():
    mainDirectory='/mnt/lustre/koa/koastore/torri_group/air_directory/Projects/DCI-Project/'
    mainCodeDirectory=os.path.join(mainDirectory,"Code/CodeFiles/")
    scratchDirectory='/mnt/lustre/koa/scratch/air673/'
    codeDirectory=os.getcwd()
    return mainDirectory,mainCodeDirectory,scratchDirectory,codeDirectory

[mainDirectory,mainCodeDirectory,scratchDirectory,codeDirectory] = GetDirectories()

In [ ]:
####################################
#LOADING CLASSES

In [ ]:
#IMPORT CLASSES (from current directory)
sys.path.append(os.path.join(mainCodeDirectory,"2_Variable_Calculation"))
from CLASSES_Variable_Calculation import ModelData_Class, SlurmJobArray_Class, DataManager_Class

In [ ]:
#data loading class
ModelData = ModelData_Class(mainDirectory, scratchDirectory, simulationNumber=2)
#data manager class
DataManager = DataManager_Class(mainDirectory, scratchDirectory, ModelData.res, ModelData.t_res, ModelData.Nz_str,
                                ModelData.Np_str, dataType="LagrangianArrays", dataName="PROCESSED_Lagrangian_Binary_Array",
                                dtype='int8')

In [ ]:
#IMPORT CLASSES
sys.path.append(os.path.join(mainCodeDirectory,"3_Project_Algorithms","2_Tracking_Algorithms"))
from CLASSES_TrackingAlgorithms import SlurmJobArray_Class

In [ ]:
import sys
path=os.path.join(mainCodeDirectory,'Functions/')
sys.path.append(path)

import NumericalFunctions
from NumericalFunctions import * # import NumericalFunctions 
import PlottingFunctions
from PlottingFunctions import * # import PlottingFunctions


# # Get all functions in NumericalFunctions
# import inspect
# functions = [f[0] for f in inspect.getmembers(NumericalFunctions, inspect.isfunction)]
# functions

# # Get all functions in NumericalFunctions
# import inspect
# functions = [f[0] for f in inspect.getmembers(PlottingFunctions, inspect.isfunction)]
# functions

In [ ]:
##############################
#SETUP

In [ ]:
################################
#JOB ARRAY SETUP
################################
#*#*
# how many total jobs are being run? i.e. array=1-100 ==> num_jobs=100
if ModelData.Np_str=='1e6': #1M parcels
    num_jobs=60  
    num_slurm_jobs=20 #this is the number that goes into sbatch script
elif ModelData.Np_str=='50e6': #50M parcels
    num_jobs=1200 
    num_slurm_jobs=150 #this is the number that goes into sbatch script
##############################

In [ ]:
##############################################
#DATA LOADING FUNCTIONS

In [ ]:
#INITIALIZE DATA FUNCTION
###############################################################
def InitiateArray(out_file, vars, t_chunk_size, p_chunk_size, t_size=None, p_size=None):
    if t_size is None:
        t_size = ModelData.Ntime  # Number of timesteps
    if p_size is None:
        p_size = ModelData.Np  # Number of parcel indexes

    with h5py.File(out_file, 'w') as f:
        for var_name in vars:
            if var_name not in f:
                # Set dtype conditionally
                if var_name in ['Z', 'Y', 'X']:
                    dtype = np.uint16
                elif var_name in ['A_g','A_c','PROCESSED_A_g','PROCESSED_A_c']:
                    dtype = np.bool_
                else:
                    dtype = np.float32  # or whatever your default is

                f.create_dataset(
                    var_name,
                    shape=(t_size, p_size),
                    chunks=(t_chunk_size, p_chunk_size),
                    dtype=dtype
                )
def InitiateArray_Job(Np,out_file, vars, t_chunk_size, p_chunk_size, t_size=None, p_size=None):
    if t_size is None:
        t_size = ModelData.Ntime  # Number of timesteps
    if p_size is None:
        p_size = Np # Number of vertical levels

    with h5py.File(out_file, 'w') as f:
        for var_name in vars:
            if var_name not in f:
                # Set dtype conditionally
                if var_name in ['Z', 'Y', 'X']:
                    dtype = np.uint16
                elif var_name in ['A_g','A_c','PROCESSED_A_g','PROCESSED_A_c']:
                    dtype = np.bool_
                else:
                    dtype = np.float32  # or whatever your default is

                f.create_dataset(
                    var_name,
                    shape=(t_size, p_size),
                    chunks=(t_chunk_size, p_chunk_size),
                    dtype=dtype
                )

In [ ]:
# MULTIPLE FILES

import os
import re
import pandas as pd
import xarray as xr
from glob import glob

def TestingTimes(files):
    print(files)
    for f in files:
        m = re.search(r'_(\d+-\d+-\d+)\.h5$', f)
        if m:
            print(m.group(1))
def OpenMultipleSingleTimes_LagrangianArray(directory, ModelData, pattern="Lagrangian_Binary_Array_*.h5"):
    """
    Load a sequence of Lagrangian .h5 files (each a single timestep)
    into one xarray.Dataset with dimensions (time, p),
    enforcing time order from ModelData.timeStrings.
    """
    # --- Find all available files
    files_all = glob(os.path.join(directory, pattern))
    if not files_all:
        raise FileNotFoundError(f"No files found in {directory} matching {pattern}")

    # --- Build the correctly ordered list according to ModelData.timeStrings
    files = []
    for t in ModelData.timeStrings:
        time_pattern = f"_{t}.h5"
        matched = [f for f in files_all if f.endswith(time_pattern)]
        if matched:
            files.append(matched[0])
        else:
            print(f"Missing file for time {t}")

    #####
    # TestingTimes(files) 
    #####
    
    # --- Convert ModelData.timeStrings (like ['0-00-00', '0-05-00', ...]) to pandas datetime
    #     using an arbitrary date
    times = pd.to_datetime([t.replace('-', ':') for t in ModelData.timeStrings], format="%H:%M:%S")

    # --- Open and concatenate along time
    ds = xr.open_mfdataset(
        files,
        engine="h5netcdf",
        phony_dims="sort",
        combine="nested",
        concat_dim="time",
    )

    # --- Rename the phony dimension to 'p'
    if "phony_dim_0" in ds.dims:
        ds = ds.rename({"phony_dim_0": "p"})

    # --- Assign your correct time coordinate
    ds = ds.assign_coords(time=times)

    return ds, files

In [ ]:
# Reading Back Data Later
##############
def make_data_dict(var_names,start_job,end_job):

    data_dict = {var_name: Lagrangian_Binary_Array[var_name].isel(p=slice(start_job, end_job)).data.compute()
                 for var_name in var_names}
    return data_dict

In [ ]:
def GetArrays(start_job,end_job):    
    var_names = ['A_g', 'A_c']
    data_dict = make_data_dict(var_names,start_job,end_job)
    A_g, A_c = (data_dict[k] for k in var_names)
    return A_g, A_c

In [ ]:
################################
#FUNCTIONS

In [ ]:
def extend_idxs(f,case):
    out=np.sort(np.add.outer(f, np.arange(case)).ravel())

    # #OLD METHOD (SLOW)
    # if np.any(f)==True:
    #     out=np.sort(np.concatenate([np.arange(idx, idx + case-1+1) for idx in f]))
    # else: 
    #     out=f
    return out

def find_sandwiched_patterns(changes, case):
    arr=changes
    
    window_size = case + 1  # e.g., for case=2, window_size = 3
    # The interior zeros count is (window_size - 2) which is case - 1
    pattern1 = np.array([-1] + [0]*(case - 1) + [1])
    pattern2 = np.array([1] + [0]*(case - 1) + [-1])
    # print(pattern1,pattern2)
    
    # Manually construct sliding windows
    windows = np.array([arr[i:i + window_size] for i in range(len(arr) - window_size + 1)])
    # print("Sliding windows:\n", windows) #TESTING
    
    #THE ALGORITHM
    turb_d=[]
    turb_e=[]
    count=0;max_iter=ModelData.Ntime;
    while np.any(((windows == pattern1) | (windows == pattern2)).all(axis=1)):
        count+=1; 
        if count>=max_iter: 
            print(count)
            break
        
        next_ind = np.where(((windows == pattern1) | (windows == pattern2)).all(axis=1))[0][0]
        
        if (windows[next_ind] == pattern1).all():
            turb_d.append(next_ind)
        elif (windows[next_ind] == pattern2).all(): 
            turb_e.append(next_ind) #append to list
    
        windows[0:next_ind+(case)+1,:] = 0 #removes from windows
    
    turb_d=np.array(turb_d,dtype=int); turb_e=np.array(turb_e,dtype=int)

    #EXTEND REST OF INDEXES TO PROCESS
    turb_d=extend_idxs(turb_d,case=case)
    turb_e=extend_idxs(turb_e,case=case)
    return turb_d,turb_e

In [ ]:
# # TESTING
# changes = np.array([0,0,0,-1,1,0,0,-1,0,0,0,1,-1,0,0])
# [a,b] = find_sandwiched_patterns(changes, case=1) #<=1 in a row timesteps are removed
# print("Case matches at indices:", a,b)

# changes = np.array([0,0,0,-1,0,1,0,0,-1,0,0,1,0,-1,0,0])
# [a,b] = find_sandwiched_patterns(changes, case=2) #<=2 in a row timesteps are removed
# print("Case matches at indices:", a,b)

# changes = np.array([0,0,0,-1,0,0,1,0,0,0,0,1,0,0,-1,0,0])
# [a,b] = find_sandwiched_patterns(changes, case=3) #<=3 in a row timesteps are removed
# print("Case matches at indices:", a,b)

# changes = np.array([0,0,0,-1,0,0,0,1,0,0,0,0,1,0,0,-1,0,0])
# [a,b] = find_sandwiched_patterns(changes, case=4) #<=4 in a row timesteps are removed
# print("Case matches at indices:", a,b)

# changes = np.array([0,0,0,-1,0,0,0,0,1,0,0,0,0,1,0,0,-1,0,0])
# [a,b] = find_sandwiched_patterns(changes, case=5) #<=5 in a row timesteps are removed
# print("Case matches at indices:", a,b)

In [ ]:
###### (amount of time inside/outside of cloud to count as entrainment/detrainment)
mins_thresh=5 #5 mins
######

t_per_mins=1/((ModelData.time[1]-ModelData.time[0])/1e9/60).item() #timesteps per minute (<=1)
def get_changes(B):
    changes = np.diff(np.concatenate(([B[0]], B)))  # Add 0s to detect edges
    return changes
def PreProcessing(A,p):
    B = A[:,p]*1

    # Find the changes in the array
    changes=get_changes(B)
    # print(f'B = {B}'); print(f'changes = {changes}') 

    #Determining the Case Number
    case=int(t_per_mins*mins_thresh)
    
    if case>1:
        for case_ind in np.arange(case,0,-1): 
        # for case_ind in [case]:
            #Calling Algorithm and Correcting Parcel Data
            [turb_d,turb_e]=find_sandwiched_patterns(changes, case=case_ind)
            B[turb_d]=1
            B[turb_e]=0     
            changes=get_changes(B)
            # print(B)
    elif case==1:
        #Calling Algorithm and Correcting Parcel Data
        [turb_d,turb_e]=find_sandwiched_patterns(changes, case=case)
        B[turb_d]=1
        B[turb_e]=0
    return B

In [ ]:
# #TESTING #CASE COUNTDOWN
# #shows the removal process (same for cases less than applicable case)
# B=np.array([1,1,1,1,0,0,0,1,1,1,1,0,0,0,1,0,0,1,1,1,0,0,0,1])
#                     #,#,#,        #,#,#       #,#,#
# print(B)

# #APPLYING
# changes=get_changes(B)
# [turb_d,turb_e]=find_sandwiched_patterns(changes,case=3)
# print(turb_d,turb_e)

# B[turb_d]=1
# B[turb_e]=0
# print(B)

# #APPLYING
# changes=get_changes(B)
# [turb_d,turb_e]=find_sandwiched_patterns(changes,case=2)
# print(turb_d,turb_e)

# B[turb_d]=1
# B[turb_e]=0
# print(B)

# #APPLYING
# changes=get_changes(B)
# [turb_d,turb_e]=find_sandwiched_patterns(changes,case=1)
# print(turb_d,turb_e)

# B[turb_d]=1
# B[turb_e]=0
# print(B)

In [ ]:
#RUNNING
def Apply(A_g,A_c, Np):
    PROCESSED_A_g = A_g.copy()
    PROCESSED_A_c = A_c.copy()
    for p in np.arange(Np):
        if np.mod(p,1e3)==0: print(f"{p}/{Np}")
        out1=PreProcessing(PROCESSED_A_g,p); PROCESSED_A_g[:,p]=out1
        out2=PreProcessing(PROCESSED_A_c,p); PROCESSED_A_c[:,p]=out2
    return PROCESSED_A_g,PROCESSED_A_c

def CheckDifferences(A_g,A_c, PROCESSED_A_g,PROCESSED_A_c):
    diff_g = np.sum(A_g != PROCESSED_A_g)
    diff_c = np.sum(A_c != PROCESSED_A_c)
    print(f"Differences: A_g={diff_g}, A_c={diff_c}")

#SAVING
def Save(Np,PROCESSED_A_g,PROCESSED_A_c,job_id):
    out_file=DataManager.outputDataDirectory+f'/PROCESSED_Lagrangian_Binary_Array_{ModelData.res}_{ModelData.t_res}_{ModelData.Np_str}_{job_id}.h5'
    print(f"Saving as {out_file}","\n")
    
    vars=['PROCESSED_A_g','PROCESSED_A_c']
    InitiateArray_Job(Np,out_file,vars,t_chunk_size=50,p_chunk_size=1000)
    
    with h5py.File(out_file, 'a') as f: 
        f['PROCESSED_A_g'][:]=PROCESSED_A_g
        f['PROCESSED_A_c'][:]=PROCESSED_A_c

In [ ]:
####################################
#RUNNING
running=True #KEEP TRUE WHEN JOB ARRAY IS RUNNING
running=False 

In [ ]:
if running:
    #Loading Lagrangian Binary Array All Times
    directory = f"/mnt/lustre/koa/koastore/torri_group/air_directory/Projects/DCI-Project/Code/OUTPUT/Variable_Calculation/LagrangianArrays/{ModelData.res}_{ModelData.t_res}_{ModelData.Nz_str}nz/Lagrangian_Binary_Array/"
    
    print("Loading Combined Lagrangian_Binary_Array")
    Lagrangian_Binary_Array,files = OpenMultipleSingleTimes_LagrangianArray(directory, ModelData)
    print("Done","\n")

In [ ]:
if running:
    #Running
    [start_slurm_job,end_slurm_job]=SlurmJobArray_Class.StartSlurmJobArray(num_jobs=num_jobs,num_slurm_jobs=num_slurm_jobs,ISRUN=True) #if ISRUN is False, then will not run using slurm_job_array
    
    print(f"Running on Slurm_Jobs for Slurm_Job_Ids: {(start_slurm_job,end_slurm_job-1)}")
    
    job_id_list=np.arange(start_slurm_job,end_slurm_job)
    for job_id in job_id_list:
        if job_id % 1 ==0: print(f"current job_id = {job_id}\n")
        [start_job,end_job,index_adjust]=SlurmJobArray_Class.StartJobArray(ModelData, job_id, num_jobs)
        print(f'Running on Parcels {start_job}-{end_job}')
        Np=end_job-start_job
        [A_g,A_c]=GetArrays(start_job,end_job)
        [PROCESSED_A_g,PROCESSED_A_c]=Apply(A_g,A_c, Np)
        CheckDifferences(A_g,A_c, PROCESSED_A_g,PROCESSED_A_c)
        Save(Np,PROCESSED_A_g,PROCESSED_A_c,job_id)

In [ ]:
#COMBINING JOB_ARRAYS AFTER RUNNING
########################################################################
recombine=False #KEEP FALSE WHEN JOB ARRAY IS RUNNING
recombine=True 

In [ ]:
if recombine:

    #IMPORT CLASSES (from current directory)
    sys.path.append(os.path.join(mainCodeDirectory,"2_Variable_Calculation"))
    from CLASSES_Variable_Calculation import ModelData_Class, SlurmJobArray_Class
    
    #JOB ARRAY SETUP
    UsingJobArray=True
    
    def GetNumJobs(res,t_res):
        if res=='1km':
            if t_res=='5min':
                num_jobs=20
            elif t_res=='1min':
                num_jobs=100
        elif res=='250m': 
            if t_res=='1min':
                num_jobs=500
        return num_jobs
    num_jobs = GetNumJobs(ModelData.res,ModelData.t_res)
    SlurmJobArray = SlurmJobArray_Class(total_elements=ModelData.Ntime, num_jobs=num_jobs, UsingJobArray=UsingJobArray)
    start_job = SlurmJobArray.start_job; end_job = SlurmJobArray.end_job
    
    def GetNumElements():
        loop_elements = np.arange(ModelData.Ntime)[start_job:end_job]
        return loop_elements
    loop_elements = GetNumElements()

In [ ]:
# def Recombine(num_jobs):
#     dir2=dir+'Project_Algorithms/Entrainment/Processing_Out/'
#     dir3=dir+'Project_Algorithms/Entrainment/OUTPUT/'
#     out_file=dir3+f'processed_binary_arrays_{res}_{t_res}_{Np_str}.h5'
    
#     vars=['PROCESSED_A_g','PROCESSED_A_c']
#     InitiateArray(out_file,vars,t_chunk_size=50,p_chunk_size=100_000)
    
#     with h5py.File(out_file, 'r+') as f_out:
        
#         for job_id in np.arange(1,num_jobs+1):
#             if np.mod(job_id,5)==0: print(f"job_id = {job_id}")
#             [a,b,_] = StartJobArray(num_jobs,job_id)
        
#             in_file=DataManager.outputDataDirectory+f'processed_binary_arrays_{res}_{t_res}_{Np_str}_{job_id}.h5'
    
#             with h5py.File(in_file, 'r') as f_in: 
#                 for var in vars:
#                     f_out[var][:,a:b]=f_in[var][:]

In [ ]:
# def Recombine_Dask():
#     #DASK-ENABLED
#     import glob, re
#     import dask
#     from dask.diagnostics import ProgressBar
    
#     def recombine_func(in_files,out_file):
#         # matching_files = sorted(glob.glob(in_files))
#         matching_files = sorted(
#         glob.glob(in_files),
#         key=lambda f: int(re.search(r'_(\d+)\.h5$', f).group(1))
#     )
        
#         print(f'recombining {len(matching_files)} files')
#         print('Forming Dask Delayed Object')
#         # out=xr.open_mfdataset(matching_files,engine='h5netcdf',concat_dim='phony_dim_1',combine='nested',phony_dims='sort')
#         if ModelData.Np_str == '1e6':
#             dim_1_length = 100_000
#         elif ModelData.Np_str == '50e6':
#             dim_1_length = 500_000
#         chunks={'phony_dim_0': -1, 'phony_dim_1':dim_1_length}
#         out = xr.open_mfdataset(
#             matching_files,
#             engine='h5netcdf',
#             concat_dim='phony_dim_1',
#             combine='nested',
#             phony_dims='sort',
#             chunks=chunks,
#             parallel=True
#         ) 
#         print('RUNNING')
#         with ProgressBar():
#             delayed_write=out.to_netcdf(out_file, engine='h5netcdf', compute=False)
#             delayed_write.compute()
#         return out

#     ####################################################################################
#     in_files=DataManager.outputDataDirectory+f'/PROCESSED_Lagrangian_Binary_Array_{ModelData.res}_{ModelData.t_res}_{ModelData.Np_str}_*.h5'
#     out_file=DataManager.outputDataDirectory+f'/PROCESSED_Lagrangian_Binary_Array_{ModelData.res}_{ModelData.t_res}_{ModelData.Np_str}.h5'
#     out = recombine_func(in_files,out_file)
#     print("\n",f"Saved to {out_file}")
#     return out

In [ ]:
# def Recombine_ByTime_V1():
#     import glob, re, os
#     import xarray as xr
#     from tqdm import tqdm

#     in_files = (
#         DataManager.outputDataDirectory
#         + f'/PROCESSED_Lagrangian_Binary_Array_{ModelData.res}_'
#           f'{ModelData.t_res}_{ModelData.Np_str}_*.h5'
#     )

#     matching_files = sorted(
#         glob.glob(in_files),
#         key=lambda f: int(re.search(r'_(\d+)\.h5$', f).group(1))
#     )

#     parcelChunk = 2_000_000 if ModelData.Np_str == "50e6" else 200_000
#     outDir = DataManager.outputDataDirectory
#     os.makedirs(outDir, exist_ok=True)

#     # ---- OPEN ALL DATASETS ONCE
#     print("Opening all files")
#     datasets = [
#         xr.open_dataset(
#             f,
#             engine="h5netcdf",
#             phony_dims="sort",
#             chunks={"phony_dim_1": parcelChunk}
#         )
#         for f in matching_files
#     ]

#     # ---- LOOP OVER TIME
#     print("Looping over all times")
#     for t in tqdm(loop_elements, desc="Recombining time steps"):
#         slices = [ds.isel(phony_dim_0=t) for ds in datasets]

#         ds_t = xr.concat(slices, dim="phony_dim_1")

#         outFile = (
#             f"{outDir}/PROCESSED_Lagrangian_Binary_Array_"
#             f"{ModelData.res}_{ModelData.t_res}_{ModelData.Nz_str}nz_"
#             f"{ModelData.timeStrings[t]}.h5"
#         )

#         ds_t.to_netcdf(outFile, engine="h5netcdf")

#         del ds_t, slices

#     # ---- CLEANUP
#     for ds in datasets:
#         ds.close()


In [ ]:
def Recombine_ByTime_V2():
    import glob, re, os
    import xarray as xr
    from tqdm import tqdm

    in_files = (
        DataManager.outputDataDirectory
        + f'/PROCESSED_Lagrangian_Binary_Array_{ModelData.res}_'
          f'{ModelData.t_res}_{ModelData.Np_str}_*.h5'
    )

    matching_files = sorted(
        glob.glob(in_files),
        key=lambda f: int(re.search(r'_(\d+)\.h5$', f).group(1))
    )

    parcelChunk = 2_000_000 if ModelData.Np_str == "50e6" else 200_000
    outDir = DataManager.outputDataDirectory
    os.makedirs(outDir, exist_ok=True)

    # ---- OPEN & CONCAT ONCE (KEY CHANGE)
    print("Opening and concatenating all files (once)")
    ds_all = xr.open_mfdataset(
        matching_files,
        engine="h5netcdf",
        phony_dims="sort",
        combine="nested",
        concat_dim="phony_dim_1",      # parcel dimension
        chunks={"phony_dim_1": parcelChunk},
        parallel=False
    )

    # ---- LOOP OVER TIME (FAST)
    print("Looping over all times")
    for t in tqdm(loop_elements, desc="Recombining time steps"):
        ds_t = ds_all.isel(phony_dim_0=t)

        outFile = (
            f"{outDir}/PROCESSED_Lagrangian_Binary_Array_"
            f"{ModelData.res}_{ModelData.t_res}_{ModelData.Nz_str}nz_"
            f"{ModelData.timeStrings[t]}.h5"
        )

        # ds_t.to_netcdf(outFile, engine="h5netcdf")
        with h5py.File(outFile, "w") as f_data:
            for varName, da in ds_t.data_vars.items():
                f_data.create_dataset(
                    varName,
                    data=da.data,        
                    dtype="bool",
                    compression=None       # IMPORTANT for speed
                )
        
                # minimal metadata (optional)
                for attr, val in da.attrs.items():
                    f_data[varName].attrs[attr] = val
        del ds_t

    ds_all.close()


In [ ]:
if recombine==True:
    # Recombine(num_jobs=num_jobs)
    # out=Recombine_Dask()
    Recombine_ByTime_V2()

In [ ]:
#################
#TESTING

In [ ]:
# #Testing Equivalent between Recombine_Dask() and Recombine_ByTime()

# lst1,lst2=[],[]
# for t in tqdm(range(ModelData.Ntime)):
    
#     path1=f"/mnt/lustre/koa/koastore/torri_group/air_directory/Projects/DCI-Project/Code/OUTPUT/Variable_Calculation/LagrangianArrays/1km_5min_34nz/PROCESSED_Lagrangian_Binary_Array/backup_Recombine_Dask/PROCESSED_Lagrangian_Binary_Array_1km_5min_34nz_{ModelData.timeStrings[t]}.h5"
#     path2=f"/mnt/lustre/koa/koastore/torri_group/air_directory/Projects/DCI-Project/Code/OUTPUT/Variable_Calculation/LagrangianArrays/1km_5min_34nz/PROCESSED_Lagrangian_Binary_Array/PROCESSED_Lagrangian_Binary_Array_1km_5min_34nz_{ModelData.timeStrings[t]}.h5"
    
#     ds1 = xr.open_dataset(path1, engine="h5netcdf", phony_dims="sort")
#     ds2 = xr.open_dataset(path2, engine="h5netcdf", phony_dims="sort")
    
#     varName1="PROCESSED_A_c"
#     varName2="PROCESSED_A_g"
    
#     a=np.all(ds1[varName1].data==ds2[varName1].data)
#     lst1.append(a)
    
#     b=np.all(ds1[varName2].data==ds2[varName2].data)
#     lst2.append(b)

# plt.plot(lst1)
# plt.plot(lst2)